In [83]:
from irsx.xmlrunner import XMLRunner
import os, requests, re, time, numpy as np, pandas as pd, pickle, json
from IPython.display import clear_output
from pandas.io.json import json_normalize
from collections import OrderedDict

def get_node_names(parent):
    node_names = []
    if type(parent) == list:
        
    for item in parent.items():
        if item[1] != None:
            if type(item[1]) == str:
                node_names.append(item[0])
            
            elif type(item[1]) == list:
                try:
                    child_list_node_names = get_node_names(item[1][0])
                    for child in child_list_node_names:
                        string_name = item[0] + '.' + child
                        node_names.append(string_name)
                except AttributeError:
                    pass
            
            else:
                child_node_names = get_node_names(item[1])
                for child in child_node_names:
                    string_name = item[0] + '.' + child
                    node_names.append(string_name)
            
    return node_names

def get_node_values(parent):
    value_list = []
    for item in parent.items():
        if item[1] != None:
            if type(item[1]) == str:
                value_list.append(item[1])
            
            elif type(item[1]) == list:
                try:
                    child_values = get_node_values(item[1][0])
                    for value in child_values:
                        value_list.append(value)
                except AttributeError:
                    print('encountered an error')
            
            else:
                child_values = get_node_values(item[1])
                for value in child_values:
                    value_list.append(value)
    
    return value_list

In [2]:
os.chdir('Filing Annual Index')

In [3]:
os.listdir()

['index_2014.csv',
 'index_2015.csv',
 'index_2016.csv',
 'index_2017.csv',
 'index_2018.csv',
 'index_2019.csv']

In [4]:
csv_frame = pd.read_csv("index_2015.csv")

In [5]:
csv_frame.head(12)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
0,12932528,EFILE,390963472,201412,10/6/2015,LUTHERDALE BIBLE CAMP INC,990,93493239006145,201542399349300614
1,12932529,EFILE,43575807,201506,10/6/2015,PLYMOUTH REGIONAL ECONOMIC DEVELOPMENT FOUNDAT...,990,93493239006195,201542399349300619
2,12932531,EFILE,942796496,201412,10/6/2015,INDOCHINESE HOUSING DEVELOPMENT CORP,990,93493239006295,201542399349300629
3,12932532,EFILE,592856392,201506,10/6/2015,HABITAT FOR HUMANITY OF EAST POLK COUNTY INC,990,93493239006345,201542399349300634
4,12932534,EFILE,953006805,201506,10/6/2015,PLEASANTVIEW INDUSTRIES INC,990,93493239007195,201542399349300719
5,12932535,EFILE,270964918,201412,10/6/2015,ROCKFORD AREA STRATEGIC INITIATIVE,990,93493239007245,201542399349300724
6,12932537,EFILE,752772633,201412,10/6/2015,NETWORK 211 INC,990,93493239007395,201542399349300739
7,12932539,EFILE,42488505,201412,10/6/2015,SCHWAMB MILL PRESERVATION TRUST,990,93493236001025,201522369349300102
8,12932540,EFILE,262082745,201412,10/6/2015,MOAB FREE HEALTH CLINIC,990,93493236001125,201522369349300112
9,12932541,EFILE,231672348,201412,10/6/2015,UNITED WAY OF PENNSYLVANIA,990,93493236001175,201522369349300117


In [6]:
csv_frame.shape

(261032, 9)

In [7]:
habitat_frame = csv_frame[csv_frame['TAXPAYER_NAME'].str.contains(r'(?i)(Habitat for Humanity)')]

C:\Users\timot\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [8]:
habitat_frame.head(12)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
3,12932532,EFILE,592856392,201506,10/6/2015,HABITAT FOR HUMANITY OF EAST POLK COUNTY INC,990,93493239006345,201542399349300634
751,12934932,EFILE,431711135,201506,10/7/2015,HABITAT FOR HUMANITY INTL JEFFERSON CO,990,93493265004315,201512659349300431
2714,12176745,EFILE,232604510,201406,1/27/2015,WYOMING VALLEY HABITAT FOR HUMANITY,990,93493009001315,201510099349300131
2727,12176759,EFILE,561668353,201406,1/27/2015,HABITAT FOR HUMANITY INTERNATIONAL HAYWOOD,990,93493009004115,201510099349300411
2836,12176794,EFILE,541375465,201406,1/27/2015,HABITAT FOR HUMANITY IN THE ROANOKE VALLEY,990,93493009006255,201500099349300625
2855,12176823,EFILE,383060981,201406,1/27/2015,MECOSTA COUNTY HABITAT FOR HUMANITY,990,93493009011005,201500099349301100
2968,12176862,EFILE,581674696,201406,1/27/2015,MACON AREA HABITAT FOR HUMANITY,990,93493015008115,201510159349300811
2981,12176884,EFILE,742524800,201406,1/27/2015,HABITAT FOR HUMANITY INTERNATIONAL INC KERR CO...,990,93493012015725,201520129349301572
3348,12177025,EFILE,680085804,201406,1/27/2015,SACRAMENTO HABITAT FOR HUMANITY,990,93493015014815,201510159349301481
3679,12176200,EFILE,460410933,201406,1/27/2015,BLACK HILLS AREA HABITAT FOR HUMANITY,990,93493012003095,201540129349300309


In [9]:
habitat_frame.shape

(599, 9)

# Get a sample of 25 and replace

In [10]:
sample_frame = habitat_frame.sample(n=25, random_state = 1)

In [11]:
sample_frame

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
195368,12633516,EFILE,621577879,201412,7/2/2015,OBION COUNTY HABITAT FOR HUMANITY,990,93493124011175,201521249349301117
76727,12696952,EFILE,752239189,201412,7/21/2015,FORT WORTH AREA HABITAT FOR HUMANIT DBA TRINIT...,990,93493187012085,201531879349301208
151626,12865781,EFILE,911755363,201412,9/16/2015,HABITAT FOR HUMANITY,990,93493219006205,201502199349300620
234446,12216589,EFILE,570785521,201406,2/23/2015,CENTRAL SC HABITAT FOR HUMANITY,990,93493042014365,201510429349301436
28914,12274227,EFILE,61448284,201406,3/19/2015,MIDDLESEX HABITAT FOR HUMANITY OF CONNECTICUT INC,990,93493068000185,201530689349300018
5339,12183637,EFILE,752391661,201406,1/29/2015,HABITAT FOR HUMANITY INTERNATIONAL GRAYSON COU...,990,93493013002315,201510139349300231
180105,12191465,EFILE,760273510,201406,2/4/2015,HABITAT FOR HUMANITY NORTHWEST HARRIS COUNTY,990,93493021005435,201530219349300543
57862,12656232,EFILE,581913989,201406,7/9/2015,HABITAT FOR HUMANITY INTERNATIONAL INC TROUP C...,990,93493175005255,201501759349300525
12325,12203845,EFILE,561733643,201406,2/17/2015,OUR TOWNS OF NORTH MECKLENBURG - SOUTH IREDELL...,990,93493036003485,201530369349300348
24115,12247175,EFILE,650108974,201406,3/10/2015,HABITAT FOR HUMANITY OF GREATER MIAMI INC,990,93493048013265,201510489349301326


In [12]:
id_array = np.array(sample_frame.OBJECT_ID)

In [13]:
id_array

array([201521249349301117, 201531879349301208, 201502199349300620,
       201510429349301436, 201530689349300018, 201510139349300231,
       201530219349300543, 201501759349300525, 201530369349300348,
       201510489349301326, 201531349349302098, 201501299349300610,
       201542399349300634, 201521289349301232, 201521259349301157,
       201520269349300232, 201540479349301814, 201540489349200424,
       201520359349200607, 201542399349300804, 201530139349301603,
       201521359349306662, 201510099349301266, 201520759349301107,
       201501349349304070], dtype=int64)

# Start using IRSx by calling `XMLRunner`  Use the object_id at the zero index in id_array

In [14]:
my_runner = XMLRunner()
test_id = id_array[0]

## experiment with what is returned. Start with `run_sked()`

In [15]:
my_runner.run_sked(test_id, 'IRS990')

##### cool... this time assign it to an object and print it out

In [16]:
test_filing = my_runner.run_sked(test_id, 'IRS990')

In [17]:
test_filing

### ok... just auto-tab complete on it and see what methods are available...

In [18]:
test_filing.get_result() # <-- found this!

[{'schedule_name': 'IRS990',
  'groups': {'PrgSrvcAccmActyOthr': [{'object_id': 201521249349301117,
     'ein': '621577879',
     'Dsc': 'EXPENSES INCURRED IN BUILDING HOUSES FOR QUALIFIED RECIPIENTS'}],
   'SttsWhrCpyOfRtrnIsFldCd': [{'object_id': 201521249349301117,
     'ein': '621577879',
     'SttsWhrCpyOfRtrnIsFldCd': 'TN'}],
   'Frm990PrtVIISctnA': [{'object_id': 201521249349301117,
     'ein': '621577879',
     'PrsnNm': 'LAURA ARCHER',
     'TtlTxt': 'DIRECTOR',
     'IndvdlTrstOrDrctrInd': 'X',
     'RprtblCmpFrmOrgAmt': '0',
     'RprtblCmpFrmRltdOrgAmt': '0',
     'OthrCmpnstnAmt': '0'},
    {'object_id': 201521249349301117,
     'ein': '621577879',
     'PrsnNm': 'CHARLES CALLIS',
     'TtlTxt': 'DIRECTOR',
     'IndvdlTrstOrDrctrInd': 'X',
     'RprtblCmpFrmOrgAmt': '0',
     'RprtblCmpFrmRltdOrgAmt': '0',
     'OthrCmpnstnAmt': '0'},
    {'object_id': 201521249349301117,
     'ein': '621577879',
     'PrsnNm': 'BUTCH FUSSELL',
     'TtlTxt': 'DIRECTOR',
     'IndvdlTrstO

# can also use the method `run_filing` from the XMLRunner()

In [19]:
test_filing_2 = my_runner.run_filing(test_id)

In [20]:
# see available schedules
test_filing_2.schedules

['ReturnHeader990x',
 'IRS990',
 'IRS990ScheduleA',
 'IRS990ScheduleD',
 'IRS990ScheduleO']

In [21]:
result = test_filing_2.get_result()

In [22]:
result

[{'schedule_name': 'ReturnHeader990x',
  'groups': {},
  'schedule_parts': {'skedd_part_x': {'object_id': 201521249349301117,
    'ein': '621577879',
    'RtrnTs': '2015-05-04T14:20:23-05:00',
    'TxPrdEndDt': '2014-12-31',
    'PrprrFrmEIN': '621110839',
    'BsnssNmLn1Txt': 'OBION COUNTY HABITAT FOR HUMANITY',
    'AddrssLn1Txt': 'P O BOX 535',
    'CtyNm': 'UNION CITY',
    'SttAbbrvtnCd': 'TN',
    'ZIPCd': '38281',
    'RtrnCd': '990',
    'TxPrdBgnDt': '2014-01-01',
    'EIN': '621577879',
    'BsnssNmCntrlTxt': 'OBIO',
    'PrsnNm': 'WILLIE BELL',
    'PrsnTtlTxt': 'PRESIDENT',
    'SgntrDt': '2015-04-27',
    'PrprrPrsnNm': 'CRAIG ATWILL',
    'PTIN': 'P00307042',
    'PhnNm': '7318853661',
    'PrprtnDt': '2015-04-27',
    'TxYr': '2014'}},
  'csv_line_array': []},
 {'schedule_name': 'IRS990',
  'groups': {'PrgSrvcAccmActyOthr': [{'object_id': 201521249349301117,
     'ein': '621577879',
     'Dsc': 'EXPENSES INCURRED IN BUILDING HOUSES FOR QUALIFIED RECIPIENTS'}],
   'SttsWh

### Let's use a different filing...

In [23]:
test_id = id_array[4]
test_filing = my_runner.run_filing(test_id)

In [24]:
test_filing.schedules

['ReturnHeader990x',
 'IRS990',
 'IRS990ScheduleA',
 'IRS990ScheduleB',
 'IRS990ScheduleD',
 'IRS990ScheduleO']

In [25]:
test_filing.get_result()

[{'schedule_name': 'ReturnHeader990x',
  'groups': {},
  'schedule_parts': {'skedd_part_x': {'object_id': 201530689349300018,
    'ein': '061448284',
    'RtrnTs': '2015-03-09T11:53:05-05:00',
    'TxPrdEndDt': '2014-06-30',
    'PrprrFrmEIN': '060971998',
    'BsnssNmLn1Txt': 'MIDDLESEX HABITAT FOR HUMANITY OF',
    'AddrssLn1Txt': '34 SHUNPIKE ROAD',
    'CtyNm': 'CROMWELL',
    'SttAbbrvtnCd': 'CT',
    'ZIPCd': '06416',
    'RtrnCd': '990',
    'TxPrdBgnDt': '2013-07-01',
    'EIN': '061448284',
    'BsnssNmLn2Txt': 'CONNECTICUT INC',
    'BsnssNmCntrlTxt': 'MIDD',
    'PhnNm': '8603475689',
    'AddrssLn2Txt': 'ROOM/SUITE 24-26',
    'PrsnNm': 'JAMES KOJI',
    'PrsnTtlTxt': 'TREASURER',
    'SgntrDt': '2015-02-01',
    'DscssWthPdPrprrInd': 'true',
    'PrprrPrsnNm': 'CHRISTOPHER B CONLEY',
    'PTIN': 'P00936552',
    'PrprtnDt': '2015-03-09',
    'TxYr': '2013'}},
  'csv_line_array': []},
 {'schedule_name': 'IRS990',
  'groups': {'SttsWhrCpyOfRtrnIsFldCd': [{'object_id': 201530

In [26]:
result = test_filing.get_result()
for each in result:
    print('Schedule\'s name: {}'.format(each['schedule_name']))
    print(json.dumps(each, indent = 4))

Schedule's name: ReturnHeader990x


TypeError: Object of type int64 is not JSON serializable

In [27]:
test_id.dtype

dtype('int64')

In [28]:
id_array.dtype

dtype('int64')

In [29]:
id_array.astype

<function ndarray.astype>

### What... fine, I won't use numpy then!

In [30]:
id_list = []
for each in sample_frame['OBJECT_ID']:
    id_list.append(each)

In [31]:
type(id_list[4])

int

# new round of testing

In [32]:
test_id = id_list[4]
test_filing = my_runner.run_filing(test_id)

In [33]:
result = test_filing.get_result()
for each in result:
    print('Schedule\'s name: {}'.format(each['schedule_name']))
    print(json.dumps(each, indent = 4))

Schedule's name: ReturnHeader990x
{
    "schedule_name": "ReturnHeader990x",
    "groups": {},
    "schedule_parts": {
        "skedd_part_x": {
            "object_id": 201530689349300018,
            "ein": "061448284",
            "RtrnTs": "2015-03-09T11:53:05-05:00",
            "TxPrdEndDt": "2014-06-30",
            "PrprrFrmEIN": "060971998",
            "BsnssNmLn1Txt": "MIDDLESEX HABITAT FOR HUMANITY OF",
            "AddrssLn1Txt": "34 SHUNPIKE ROAD",
            "CtyNm": "CROMWELL",
            "SttAbbrvtnCd": "CT",
            "ZIPCd": "06416",
            "RtrnCd": "990",
            "TxPrdBgnDt": "2013-07-01",
            "EIN": "061448284",
            "BsnssNmLn2Txt": "CONNECTICUT INC",
            "BsnssNmCntrlTxt": "MIDD",
            "PhnNm": "8603475689",
            "AddrssLn2Txt": "ROOM/SUITE 24-26",
            "PrsnNm": "JAMES KOJI",
            "PrsnTtlTxt": "TREASURER",
            "SgntrDt": "2015-02-01",
            "DscssWthPdPrprrInd": "true",
           

# There we go...
# Now, onto finding repeating groups

In [34]:
print(result[1]['groups'].keys())

dict_keys(['SttsWhrCpyOfRtrnIsFldCd', 'Frm990PrtVIISctnA', 'PrgrmSrvcRvn', 'OthrExpnss'])


# 'Groups' are embedded iterables within the filing. Like board members, and other items with repeating keys

In [35]:
monroe = my_runner.run_filing('201841229349301619')

In [36]:
monroe.get_result()

[{'schedule_name': 'ReturnHeader990x',
  'groups': {},
  'schedule_parts': {'skedd_part_x': {'object_id': '201841229349301619',
    'ein': '351753977',
    'RtrnTs': '2018-05-02T14:21:25-05:00',
    'TxPrdEndDt': '2017-06-30',
    'PrprrFrmEIN': '311119890',
    'BsnssNmLn1Txt': 'Habitat for Humanity of Monroe County',
    'AddrssLn1Txt': '213 E Kirkwood Ave',
    'CtyNm': 'Bloomington',
    'SttAbbrvtnCd': 'IN',
    'ZIPCd': '47408',
    'RtrnCd': '990',
    'TxPrdBgnDt': '2016-07-01',
    'EIN': '351753977',
    'BsnssNmCntrlTxt': 'HABI',
    'PhnNm': '3175721130',
    'PrsnNm': 'Kerry Thomson',
    'PrsnTtlTxt': 'Executive Director',
    'SgntrDt': '2018-05-02',
    'DscssWthPdPrprrInd': '1',
    'PrprrPrsnNm': 'Rickey Roysdon CPA',
    'PTIN': 'P00645916',
    'IPv4AddrssTxt': '216.196.129.6',
    'IPDt': '2018-05-02',
    'IPTm': '10:52:22',
    'IPTmznCd': 'ED',
    'FlngLcnsCd': 'P',
    'AtSbmssnCrtnDvcId': '1CC12918659019CB9956EBBFB3F01E92D4F3F060',
    'AtSbmssnFlngDvcId': 'B

# lets pack one of these json dumps into a dataframe

In [58]:
monroe_990_schedule = monroe.get_result()[1]['schedule_parts']
monroe_schedule_json = json.dumps(monroe_990_schedule, indent=4)
print(monroe_schedule_json)

{
    "part_0": {
        "object_id": "201841229349301619",
        "ein": "351753977",
        "PrncplOffcrNm": "Kerry Thomson",
        "AddrssLn1Txt": "213 E Kirkwood Ave",
        "CtyNm": "Bloomington",
        "SttAbbrvtnCd": "IN",
        "ZIPCd": "47408",
        "GrssRcptsAmt": "3199769",
        "GrpRtrnFrAffltsInd": "0",
        "Orgnztn501c3Ind": "X",
        "WbstAddrssTxt": "monroecountyhabitat.org",
        "OfOrgnztnCrpInd": "X",
        "FrmtnYr": "1988",
        "LglDmclSttCd": "IN"
    },
    "part_i": {
        "object_id": "201841229349301619",
        "ein": "351753977",
        "ActvtyOrMssnDsc": "To provide simple, decent, and affordable housing for members of the community that are living in substandard housing conditions and are not able to qualify for a conventional mortgage.",
        "VtngMmbrsGvrnngBdyCnt": "9",
        "VtngMmbrsIndpndntCnt": "9",
        "TtlEmplyCnt": "0",
        "TtlVlntrsCnt": "1700",
        "TtlGrssUBIAmt": "0",
        "NtUnrltdB

# hang on, why am i taking this perfectly good dictionary and dumping it into a json string... I'm not saving them right now. No, I need to be unpacking the dictionaries into smaller flatter bits.

Found this online. I think it will help. https://towardsdatascience.com/how-to-flatten-deeply-nested-json-objects-in-non-recursive-elegant-python-55f96533103d

In [39]:
from itertools import chain, starmap

def flatten_json_iterative_solution(dictionary):
    """Flatten a nested json file"""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + '_' + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + '_'+str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    

            
    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

In [48]:
flat = flatten_json_iterative_solution(monroe_990)


In [50]:
flat

{'part_0_object_id': '201841229349301619',
 'part_0_ein': '351753977',
 'part_0_PrncplOffcrNm': 'Kerry Thomson',
 'part_0_AddrssLn1Txt': '213 E Kirkwood Ave',
 'part_0_CtyNm': 'Bloomington',
 'part_0_SttAbbrvtnCd': 'IN',
 'part_0_ZIPCd': '47408',
 'part_0_GrssRcptsAmt': '3199769',
 'part_0_GrpRtrnFrAffltsInd': '0',
 'part_0_Orgnztn501c3Ind': 'X',
 'part_0_WbstAddrssTxt': 'monroecountyhabitat.org',
 'part_0_OfOrgnztnCrpInd': 'X',
 'part_0_FrmtnYr': '1988',
 'part_0_LglDmclSttCd': 'IN',
 'part_i_object_id': '201841229349301619',
 'part_i_ein': '351753977',
 'part_i_ActvtyOrMssnDsc': 'To provide simple, decent, and affordable housing for members of the community that are living in substandard housing conditions and are not able to qualify for a conventional mortgage.',
 'part_i_VtngMmbrsGvrnngBdyCnt': '9',
 'part_i_VtngMmbrsIndpndntCnt': '9',
 'part_i_TtlEmplyCnt': '0',
 'part_i_TtlVlntrsCnt': '1700',
 'part_i_TtlGrssUBIAmt': '0',
 'part_i_NtUnrltdBsTxblIncmAmt': '0',
 'part_i_PYCntrbtnsG

# cool, let's try it on a the whole filing

In [51]:
monroe_full = monroe.get_result()

In [54]:
monroe_full

[{'schedule_name': 'ReturnHeader990x',
  'groups': {},
  'schedule_parts': {'skedd_part_x': {'object_id': '201841229349301619',
    'ein': '351753977',
    'RtrnTs': '2018-05-02T14:21:25-05:00',
    'TxPrdEndDt': '2017-06-30',
    'PrprrFrmEIN': '311119890',
    'BsnssNmLn1Txt': 'Habitat for Humanity of Monroe County',
    'AddrssLn1Txt': '213 E Kirkwood Ave',
    'CtyNm': 'Bloomington',
    'SttAbbrvtnCd': 'IN',
    'ZIPCd': '47408',
    'RtrnCd': '990',
    'TxPrdBgnDt': '2016-07-01',
    'EIN': '351753977',
    'BsnssNmCntrlTxt': 'HABI',
    'PhnNm': '3175721130',
    'PrsnNm': 'Kerry Thomson',
    'PrsnTtlTxt': 'Executive Director',
    'SgntrDt': '2018-05-02',
    'DscssWthPdPrprrInd': '1',
    'PrprrPrsnNm': 'Rickey Roysdon CPA',
    'PTIN': 'P00645916',
    'IPv4AddrssTxt': '216.196.129.6',
    'IPDt': '2018-05-02',
    'IPTm': '10:52:22',
    'IPTmznCd': 'ED',
    'FlngLcnsCd': 'P',
    'AtSbmssnCrtnDvcId': '1CC12918659019CB9956EBBFB3F01E92D4F3F060',
    'AtSbmssnFlngDvcId': 'B

In [53]:
monroe_flat = flatten_json_iterative_solution(monroe_full)

AttributeError: 'list' object has no attribute 'items'

# figured that wouldn't work... because the dictionaries are within a list.
# you know what, we don't need a single flat database. We can have multiple. So long as they have shared keys...
# now the problem is setting that up...

# side note, I still need to figure out how to normalize filings...

# Hang on, lets try the flattener on a dictionary that has 'groups'! The main section 'IRS990' has groups. Use it!

In [59]:
monroe_IRS990_flat = flatten_json_iterative_solution(monroe_full[1])

In [60]:
monroe_IRS990_flat

{'schedule_name': 'IRS990',
 'groups_SttsWhrCpyOfRtrnIsFldCd_0_object_id': '201841229349301619',
 'groups_SttsWhrCpyOfRtrnIsFldCd_0_ein': '351753977',
 'groups_SttsWhrCpyOfRtrnIsFldCd_0_SttsWhrCpyOfRtrnIsFldCd': 'IN',
 'groups_Frm990PrtVIISctnA_0_object_id': '201841229349301619',
 'groups_Frm990PrtVIISctnA_0_ein': '351753977',
 'groups_Frm990PrtVIISctnA_0_PrsnNm': 'Jennifer Hart',
 'groups_Frm990PrtVIISctnA_0_TtlTxt': 'Chair',
 'groups_Frm990PrtVIISctnA_0_AvrgHrsPrWkRt': '3.00',
 'groups_Frm990PrtVIISctnA_0_IndvdlTrstOrDrctrInd': 'X',
 'groups_Frm990PrtVIISctnA_0_OffcrInd': 'X',
 'groups_Frm990PrtVIISctnA_0_RprtblCmpFrmOrgAmt': '0',
 'groups_Frm990PrtVIISctnA_0_RprtblCmpFrmRltdOrgAmt': '0',
 'groups_Frm990PrtVIISctnA_0_OthrCmpnstnAmt': '0',
 'groups_Frm990PrtVIISctnA_1_object_id': '201841229349301619',
 'groups_Frm990PrtVIISctnA_1_ein': '351753977',
 'groups_Frm990PrtVIISctnA_1_PrsnNm': 'Ryan Hawkins',
 'groups_Frm990PrtVIISctnA_1_TtlTxt': 'Vice Chair',
 'groups_Frm990PrtVIISctnA_1_Avr

# nice, but adds the problem of renaming the keys. I mean, it makes the keys easier to understand, but makes them longer. And it doesn't follow the key standard used by the 990-xml-reader...

# this is only somewhat helping in getting the data normalized. need to get 2  or more IRS series together!
# move back to using the 'schedule_parts' key, grab all the main IRS990 sections from the sample.

In [91]:
filings = []
count = 0
for each in id_list:
    try:
        fil = my_runner.run_filing(each)
        obj_id = fil.get_result()[1]['schedule_parts']['part_0']['object_id']
        filing_IRS990_main = flatten_json_iterative_solution(fil.get_result()[1]['schedule_parts'])
        filings.append(OrderedDict(filing_IRS990_main))
        print('got data for {}, number {}'.format(obj_id, count))
    except:
        pass
    count += 1

got data for 201521249349301117, number 0
got data for 201531879349301208, number 1
got data for 201502199349300620, number 2
got data for 201510429349301436, number 3
got data for 201530689349300018, number 4
got data for 201510139349300231, number 5
got data for 201530219349300543, number 6
got data for 201501759349300525, number 7
got data for 201530369349300348, number 8
got data for 201510489349301326, number 9
Filing version 2012v3.0 isn't supported for this operation
got data for 201501299349300610, number 11
got data for 201542399349300634, number 12
got data for 201521289349301232, number 13
got data for 201521259349301157, number 14
got data for 201520269349300232, number 15
got data for 201540479349301814, number 16
got data for 201542399349300804, number 19
got data for 201530139349301603, number 20
got data for 201521359349306662, number 21
got data for 201510099349301266, number 22
got data for 201520759349301107, number 23
got data for 201501349349304070, number 24


In [85]:
filings

[OrderedDict([('part_0_object_id', 201521249349301117),
              ('part_0_ein', '621577879'),
              ('part_0_PrncplOffcrNm', 'WILLIE BELL'),
              ('part_0_AddrssLn1Txt', '104 WILMA DRIVE'),
              ('part_0_CtyNm', 'UNION CITY'),
              ('part_0_SttAbbrvtnCd', 'TN'),
              ('part_0_ZIPCd', '38261'),
              ('part_0_GrssRcptsAmt', '11354'),
              ('part_0_GrpRtrnFrAffltsInd', 'false'),
              ('part_0_Orgnztn501c3Ind', 'X'),
              ('part_0_OfOrgnztnCrpInd', 'X'),
              ('part_i_object_id', 201521249349301117),
              ('part_i_ein', '621577879'),
              ('part_i_ActvtyOrMssnDsc',
               'BUILDING HOUSES FOR QUALIFIED RECIPIENTS'),
              ('part_i_VtngMmbrsGvrnngBdyCnt', '12'),
              ('part_i_VtngMmbrsIndpndntCnt', '12'),
              ('part_i_TtlEmplyCnt', '2'),
              ('part_i_TtlVlntrsCnt', '250'),
              ('part_i_TtlGrssUBIAmt', '0'),
              ('par

# K, now get key names...

In [86]:
df = pd.DataFrame(filings)

In [87]:
df

,part_0_object_id,part_0_ein,part_0_PrncplOffcrNm,part_0_AddrssLn1Txt,part_0_CtyNm,part_0_SttAbbrvtnCd,part_0_ZIPCd,part_0_GrssRcptsAmt,part_0_GrpRtrnFrAffltsInd,part_0_Orgnztn501c3Ind,...,part_0_GrpExmptnNm,part_vi_PlcsRfrncChptrsInd,part_viii_PrsnlAmt,part_0_OfOrgnztnOthrInd,part_xii_MthdOfAccntngCshInd,part_0_AmnddRtrnInd,part_0_BsnssNmLn1Txt,part_xi_PrrPrdAdjstmntsAmt,part_ix_InfInSkdOPrtIXInd,part_v_NnddctblCntrDsclInd
0,201521249349301117,621577879,WILLIE BELL,104 WILMA DRIVE,UNION CITY,TN,38261,11354,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201531879349301208,752239189,GAGE YAGER,SAME AS ABOVE,FORT WORTH,TX,761104312,9282136,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201502199349300620,911755363,MICHAEL STEVENS,4736 BADGER ST,MOSES LAKE,WA,98837,194402,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201510429349301436,570785521,ROY KRAMER,209 S SUMTER ST,COLUMBIA,SC,29201,2760776,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201530689349300018,061448284,SARAH J BIRD,34 SHUNPIKE ROAD UNIT 24-26,CROMWELL,CT,06416,722167,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201510139349300231,752391661,DAN STENGER,901 N GRAND AVE,SHERMAN,TX,750904412,406647,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201530219349300543,760273510,WILLIAM STRANGE,13350 JONES ROAD,HOUSTON,TX,77070,3978855,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,201501759349300525,581913989,STEVE BROWN,PO BOX 327,LAGRANGE,GA,30241,674503,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,201530369349300348,561733643,NaN,NaN,NaN,NaN,NaN,3847860,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,201510489349301326,650108974,MARIO ARTECONA,3800 NW 22 AVENUE,MIAMI,FL,33142,15060417,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
df.shape

(22, 259)

# find the lenght of each IRS990 series

In [90]:
for each in filings:
    print(len(each))

188
204
201
209
196
211
215
213
220
219
215
217
221
214
185
200
218
214
220
186
214
211


# nice, so in a sample of 25 we were able to find 259 unique keys. There likely aren't that many more for the main section. Let's take a sample of, say, 300...

In [119]:
new_sample_frame = habitat_frame.sample(n = 300, random_state = 1)
new_sample_frame

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID
195368,12633516,EFILE,621577879,201412,7/2/2015,OBION COUNTY HABITAT FOR HUMANITY,990,93493124011175,201521249349301117
76727,12696952,EFILE,752239189,201412,7/21/2015,FORT WORTH AREA HABITAT FOR HUMANIT DBA TRINIT...,990,93493187012085,201531879349301208
151626,12865781,EFILE,911755363,201412,9/16/2015,HABITAT FOR HUMANITY,990,93493219006205,201502199349300620
234446,12216589,EFILE,570785521,201406,2/23/2015,CENTRAL SC HABITAT FOR HUMANITY,990,93493042014365,201510429349301436
28914,12274227,EFILE,61448284,201406,3/19/2015,MIDDLESEX HABITAT FOR HUMANITY OF CONNECTICUT INC,990,93493068000185,201530689349300018
5339,12183637,EFILE,752391661,201406,1/29/2015,HABITAT FOR HUMANITY INTERNATIONAL GRAYSON COU...,990,93493013002315,201510139349300231
180105,12191465,EFILE,760273510,201406,2/4/2015,HABITAT FOR HUMANITY NORTHWEST HARRIS COUNTY,990,93493021005435,201530219349300543
57862,12656232,EFILE,581913989,201406,7/9/2015,HABITAT FOR HUMANITY INTERNATIONAL INC TROUP C...,990,93493175005255,201501759349300525
12325,12203845,EFILE,561733643,201406,2/17/2015,OUR TOWNS OF NORTH MECKLENBURG - SOUTH IREDELL...,990,93493036003485,201530369349300348
24115,12247175,EFILE,650108974,201406,3/10/2015,HABITAT FOR HUMANITY OF GREATER MIAMI INC,990,93493048013265,201510489349301326


In [95]:
id_sample = []
for each in new_sample_frame['OBJECT_ID']:
    id_sample.append(each)

In [101]:
id_sample[:6]

[201521249349301117,
 201531879349301208,
 201502199349300620,
 201510429349301436,
 201530689349300018,
 201510139349300231]

In [102]:
filings = []
count = 0
for each in id_sample:
    try:
        fil = my_runner.run_filing(each)
        obj_id = fil.get_result()[1]['schedule_parts']['part_0']['object_id']
        filing_IRS990_main = flatten_json_iterative_solution(fil.get_result()[1]['schedule_parts'])
        filings.append(OrderedDict(filing_IRS990_main))
        print('got data for {}, number {}'.format(obj_id, count))
    except:
        pass
    count += 1
    


got data for 201521249349301117, number 0
got data for 201531879349301208, number 1
got data for 201502199349300620, number 2
got data for 201510429349301436, number 3
got data for 201530689349300018, number 4
got data for 201510139349300231, number 5
got data for 201530219349300543, number 6
got data for 201501759349300525, number 7
got data for 201530369349300348, number 8
got data for 201510489349301326, number 9
Filing version 2012v3.0 isn't supported for this operation
got data for 201501299349300610, number 11
got data for 201542399349300634, number 12
got data for 201521289349301232, number 13
got data for 201521259349301157, number 14
got data for 201520269349300232, number 15
got data for 201540479349301814, number 16
got data for 201542399349300804, number 19
got data for 201530139349301603, number 20
got data for 201521359349306662, number 21
got data for 201510099349301266, number 22
got data for 201520759349301107, number 23
got data for 201501349349304070, number 24
got d

got data for 201521359349300127, number 210
got data for 201541359349309119, number 211
got data for 201520479349302272, number 212
got data for 201520469349300107, number 213
got data for 201510159349300811, number 214
got data for 201521349349308432, number 215
got data for 201500159349301305, number 216
got data for 201511339349302326, number 218
got data for 201512679349300221, number 219
got data for 201520249349300607, number 220
got data for 201510829349300711, number 221
got data for 201511359349305881, number 222
got data for 201540639349300329, number 223
got data for 201510469349300626, number 224
got data for 201530759349301303, number 225
got data for 201512389349300736, number 226
got data for 201510479349301716, number 227
got data for 201521349349308807, number 228
got data for 201510099349300131, number 229
got data for 201540439349301214, number 230
got data for 201521319349302802, number 231
got data for 201541359349308024, number 232
got data for 201521279349300537,

# Flatten the main section of Monroe's  IRS990 data and append it

In [128]:
filings.append(flatten_json_iterative_solution(monroe_990_schedule))

In [129]:
df = pd.DataFrame(filings)

In [130]:
df

,part_0_object_id,part_0_ein,part_0_PrncplOffcrNm,part_0_AddrssLn1Txt,part_0_CtyNm,part_0_SttAbbrvtnCd,part_0_ZIPCd,part_0_GrssRcptsAmt,part_0_GrpRtrnFrAffltsInd,part_0_Orgnztn501c3Ind,...,part_v_InfInSkdOPrtVInd,part_0_OfOrgnztnAsscInd,part_vi_OthrInd,part_0_FnlRtrnInd,part_i_CntrctTrmntnInd,part_0_OthrOrgnztnDsc,part_v_Frm8886TFldInd,part_v_Frm720FldInd,part_viii_InfInSkdOPrtVIIIInd,part_vii_InfInSkdOPrtVIIInd
0,201521249349301117,621577879,WILLIE BELL,104 WILMA DRIVE,UNION CITY,TN,38261,11354,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201531879349301208,752239189,GAGE YAGER,SAME AS ABOVE,FORT WORTH,TX,761104312,9282136,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201502199349300620,911755363,MICHAEL STEVENS,4736 BADGER ST,MOSES LAKE,WA,98837,194402,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201510429349301436,570785521,ROY KRAMER,209 S SUMTER ST,COLUMBIA,SC,29201,2760776,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201530689349300018,061448284,SARAH J BIRD,34 SHUNPIKE ROAD UNIT 24-26,CROMWELL,CT,06416,722167,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,201510139349300231,752391661,DAN STENGER,901 N GRAND AVE,SHERMAN,TX,750904412,406647,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,201530219349300543,760273510,WILLIAM STRANGE,13350 JONES ROAD,HOUSTON,TX,77070,3978855,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,201501759349300525,581913989,STEVE BROWN,PO BOX 327,LAGRANGE,GA,30241,674503,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,201530369349300348,561733643,NaN,NaN,NaN,NaN,NaN,3847860,false,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,201510489349301326,650108974,MARIO ARTECONA,3800 NW 22 AVENUE,MIAMI,FL,33142,15060417,0,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
for each in df.keys():
    print(each)

part_0_object_id
part_0_ein
part_0_PrncplOffcrNm
part_0_AddrssLn1Txt
part_0_CtyNm
part_0_SttAbbrvtnCd
part_0_ZIPCd
part_0_GrssRcptsAmt
part_0_GrpRtrnFrAffltsInd
part_0_Orgnztn501c3Ind
part_0_OfOrgnztnCrpInd
part_i_object_id
part_i_ein
part_i_ActvtyOrMssnDsc
part_i_VtngMmbrsGvrnngBdyCnt
part_i_VtngMmbrsIndpndntCnt
part_i_TtlEmplyCnt
part_i_TtlVlntrsCnt
part_i_TtlGrssUBIAmt
part_i_PYCntrbtnsGrntsAmt
part_i_CYCntrbtnsGrntsAmt
part_i_PYPrgrmSrvcRvnAmt
part_i_CYPrgrmSrvcRvnAmt
part_i_PYInvstmntIncmAmt
part_i_CYInvstmntIncmAmt
part_i_PYOthrRvnAmt
part_i_CYOthrRvnAmt
part_i_PYTtlRvnAmt
part_i_CYTtlRvnAmt
part_i_CYGrntsAndSmlrPdAmt
part_i_CYBnftsPdTMmbrsAmt
part_i_PYSlrsCmpEmpBnftPdAmt
part_i_CYSlrsCmpEmpBnftPdAmt
part_i_CYTtlPrfFndrsngExpnsAmt
part_i_CYTtlFndrsngExpnsAmt
part_i_PYOthrExpnssAmt
part_i_CYOthrExpnssAmt
part_i_PYTtlExpnssAmt
part_i_CYTtlExpnssAmt
part_i_PYRvnsLssExpnssAmt
part_i_CYRvnsLssExpnssAmt
part_i_TtlAsstsBOYAmt
part_i_TtlAsstsEOYAmt
part_i_TtlLbltsBOYAmt
part_i_TtlLbltsEO

In [136]:
len(df) - df.count()

part_0_object_id                    0
part_0_ein                          0
part_0_PrncplOffcrNm               58
part_0_AddrssLn1Txt                66
part_0_CtyNm                       66
part_0_SttAbbrvtnCd                66
part_0_ZIPCd                       66
part_0_GrssRcptsAmt                 0
part_0_GrpRtrnFrAffltsInd           0
part_0_Orgnztn501c3Ind              1
part_0_OfOrgnztnCrpInd             12
part_i_object_id                    0
part_i_ein                          0
part_i_ActvtyOrMssnDsc              0
part_i_VtngMmbrsGvrnngBdyCnt        0
part_i_VtngMmbrsIndpndntCnt         0
part_i_TtlEmplyCnt                  0
part_i_TtlVlntrsCnt                45
part_i_TtlGrssUBIAmt                0
part_i_PYCntrbtnsGrntsAmt           5
part_i_CYCntrbtnsGrntsAmt           0
part_i_PYPrgrmSrvcRvnAmt           36
part_i_CYPrgrmSrvcRvnAmt            0
part_i_PYInvstmntIncmAmt           26
part_i_CYInvstmntIncmAmt            0
part_i_PYOthrRvnAmt                31
part_i_CYOth

# check to see how many datapoints are missing...

In [139]:
df.isnull().sum().sum()

20843

In [144]:
df.size - df.count().sum()

20843

# out of how many?

In [145]:
df.size

76422

# Not bad. Most of those datapoints will come from the fact that one affiliate has a special line item that the rest do not have.

# Now comes the manual part of validating line items.